In [1]:
!pip install transformers datasets torch seqeval accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 853.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.4 MB/s eta 0:00:00

In [2]:
# 1. Установка библиотек (если не установлены)
# !pip install transformers datasets torch seqeval accelerate

# 2. Импорт библиотек
from datasets import load_dataset, DatasetDict, concatenate_datasets
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification,
    pipeline
)
import numpy as np
from seqeval.metrics import classification_report
import torch

In [6]:
from datasets import Dataset
import os
import re

def load_brat_data(dir_path):
    data = {"tokens": [], "labels": []}

    for filename in os.listdir(dir_path):
        if not filename.endswith(".txt"):
            continue

        # Чтение текста
        txt_path = os.path.join(dir_path, filename)
        with open(txt_path, "r", encoding="utf-8") as f:
            text = f.read()

        # Токенизация по пробелам и пунктуации (адаптируйте под ваш случай)
        tokens = re.findall(r"\w+|[^\w\s]", text, re.UNICODE)
        labels = ["O"] * len(tokens)

        # Чтение аннотаций
        ann_path = txt_path.replace(".txt", ".ann")
        if not os.path.exists(ann_path):
            continue

        with open(ann_path, "r", encoding="utf-8") as f:
            for line in f:
                if line.startswith("T"):  # Entity
                    parts = line.strip().split("\t")
                    if len(parts) < 3:
                        continue

                    tag_info = parts[1]
                    entity_text = parts[2]

                    # Парсим информацию о позициях
                    tag_parts = tag_info.split()
                    try:
                        tag = tag_parts[0]
                        start = int(tag_parts[-2])
                        end = int(tag_parts[-1])
                    except (ValueError, IndexError):
                        continue

                    # Назначаем метки токенам
                    token_pos = 0
                    current_pos = 0
                    for i, token in enumerate(tokens):
                        token_len = len(token)
                        if current_pos >= start and current_pos + token_len <= end:
                            labels[i] = f"B-{tag}" if token_pos == 0 else f"I-{tag}"
                            token_pos += 1
                        current_pos += token_len + 1  # +1 для пробела/разделителя

        data["tokens"].append(tokens)
        data["labels"].append(labels)

    return Dataset.from_dict(data)

In [7]:
# Подключение выборки данных файловым способом
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
dataset = load_brat_data("./drive/MyDrive/NEREL-v1.1/train")

In [12]:
dataset

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 746
})

In [16]:
# 3. Загрузка и объединение датасетов
def load_and_preprocess_datasets():
    # Загрузка NEREL

    return DatasetDict({
        "train":load_brat_data("./drive/MyDrive/NEREL-v1.1/train"),
        "validation": load_brat_data("./drive/MyDrive/NEREL-v1.1/dev"),
        "test": load_brat_data("./drive/MyDrive/NEREL-v1.1/test")
    })

dataset = load_and_preprocess_datasets()

In [17]:
# 4. Создание словаря меток
label_list = sorted({tag for example in dataset["train"] for tag in example["labels"]})
label2id = {tag: i for i, tag in enumerate(label_list)}
id2label = {i: tag for i, tag in enumerate(label_list)}

In [20]:
# 5. Загрузка модели и токенизатора
from transformers import AutoTokenizer, AutoModelForTokenClassification
import os

# Проверка доступности модели
model_name = "DeepPavlov/rubert-base-cased"
try:
    # Загрузка токенизатора с явным указанием локального кэша
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        cache_dir=os.path.expanduser("~/.cache/huggingface/hub")
    )

    # Загрузка модели
    model = AutoModelForTokenClassification.from_pretrained(
        model_name,
        num_labels=len(label_list),
        id2label=id2label,
        label2id=label2id,
        cache_dir=os.path.expanduser("~/.cache/huggingface/hub")
    )

    print("Модель и токенизатор успешно загружены!")

except Exception as e:
    print(f"Ошибка при загрузке модели: {str(e)}")
    print("Попробуйте следующие решения:")
    print("1. Проверьте подключение к интернету")
    print("2. Попробуйте другую модель, например 'DeepPavlov/rubert-tiny'")
    print("3. Убедитесь, что у вас есть права на запись в кэш-директорию")

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Модель и токенизатор успешно загружены!


In [21]:
# 6. Токенизация данных с выравниванием меток
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=256,
        padding="max_length",
    )

    labels = []
    for i, doc_labels in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        doc_labels = [label2id[label] for label in doc_labels]

        label_ids = []
        current_label = -100
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label = doc_labels[word_idx]
                if word_idx != 0 and word_ids[word_idx] == word_ids[word_idx-1]:
                    label_ids.append(-100)  # Только первый подтокен получает метку
                else:
                    label_ids.append(label)
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/746 [00:00<?, ? examples/s]

Map:   0%|          | 0/94 [00:00<?, ? examples/s]

Map:   0%|          | 0/93 [00:00<?, ? examples/s]

In [22]:
# 7. Обучение модели
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./ner_results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    report_to="none",
)

def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [id2label[l] for l in label if l != -100]
        for label in labels
    ]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return classification_report(true_labels, true_predictions, output_dict=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-22-412e27868f30>:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Age,Award,City,Country,Crime,Date,Disease,District,Event,Facility,Family,Ideology,Language,Law,Location,Money,Nationality,Number,Ordinal,Organization,Penalty,Percent,Person,Product,Profession,Religion,State Or Province,Time,Work Of Art,Micro avg,Macro avg,Weighted avg
1,No log,1.259722,"{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 97}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 23}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 98}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 101}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 49}","{'precision': 0.23595505617977527, 'recall': 0.12804878048780488, 'f1-score': 0.1660079051383399, 'support': 328}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 80}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 426}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 61}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 25}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 45}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 23}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 18}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 32}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 53}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 15}","{'precision': 0.07575757575757576, 'recall': 0.01718213058419244, 'f1-score': 0.028011204481792718, 'support': 291}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 34}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3}","{'precision': 0.23503325942350334, 'recall': 0.17462932454695224, 'f1-score': 0.20037807183364842, 'support': 607}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12}","{'precision': 0.0916030534351145, 'recall': 0.028985507246376812, 'f1-score': 0.044036697247706424, 'support': 414}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 38}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 26}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 70}","{'precision': 0.19320843091334894, 'recall': 0.055220883534136546, 'f1-score': 0.08589276418532016, 'support': 2988}","{'precision': 0.022012032579171344, 'recall': 0.01202916354708022, 'f1-score': 0.01511840961039612, 'support': 2988}","{'precision': 0.09371739142055714, 'recall': 0.055220883534136546, 'f1-score': 0.0677585460686587, 'support': 2988}"
2,No log,1.020934,"{'precision': 0.4772727272727273, 'recall': 0.21649484536082475, 'f1-score': 0.2978723404255319, 'support': 97}","{'precision': 0.09090909090909091, 'recall': 0.043478260869565216, 'f1-score': 0.0588235294117647, 'support': 23}","{'precision': 0.21428571428571427, 'recall': 0.061224489795918366, 'f1-score': 0.09523809523809523, 'support': 98}","{'precision': 1.0, 'recall': 0.0297029702970297, 'f1-score': 0.05769230769230769, 'support': 101}","{'precision': 0.05263157894736842, 'recall': 0.02040816326530612, 'f1-score': 0.02941176470588235, 'support': 49}","{'precision': 0.21982758620689655, 'recall': 0.15548780487804878, 'f1-score': 0.18214285714285713, 'support': 328}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 80}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 9}","{'precision': 0.22033898305084745, 'recall': 0.03051643192488263, 'f1-score': 0.0536082474226804, 'support': 426}","{'precision': 0.2222222222222222, 'recall': 0.03278688524590164, 'f1-score': 0.05714285714285715, 'support': 61}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5}","{'precision': 0.0, 'rec

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

TrainOutput(global_step=470, training_loss=1.0249787026263297, metrics={'train_runtime': 12821.396, 'train_samples_per_second': 0.291, 'train_steps_per_second': 0.037, 'total_flos': 487569582842880.0, 'train_loss': 1.0249787026263297, 'epoch': 5.0})

In [31]:
# 8. Сохранение модели
model.save_pretrained("./drive/MyDrive/NEREL-model-test/5epoch")
tokenizer.save_pretrained("./drive/MyDrive/NEREL-model-test/5epoch")

('./drive/MyDrive/NEREL-model-test/5epoch/tokenizer_config.json',
 './drive/MyDrive/NEREL-model-test/5epoch/special_tokens_map.json',
 './drive/MyDrive/NEREL-model-test/5epoch/vocab.txt',
 './drive/MyDrive/NEREL-model-test/5epoch/added_tokens.json',
 './drive/MyDrive/NEREL-model-test/5epoch/tokenizer.json')

In [30]:
import subprocess
subprocess.run(['cp', '-r', 'ner_results', './drive/MyDrive/NEREL-model-test/5epoch'])

CompletedProcess(args=['cp', '-r', 'ner_results', './drive/MyDrive/NEREL-model-test/5epoch'], returncode=1)

In [25]:
# 9. Тестирование
nlp = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="average",
    device=0 if torch.cuda.is_available() else -1,
)

text = "Правительство России задумало вернуть налог на движимое имущество, который отменили с 1 января 2019 года, еще при предыдущем составе кабмина. Об этом заявил замминистра финансов Алексей Лавров, пишут «Ведомости». Спикер Совета Федерации Валентина Матвиенко напомнила, что ранее предупреждали о негативных последствиях налогового послабления: «Освободить от налога стоило только новое оборудование, приобретаемое компаниями, но нас не послушали, все освободили». В 2019 году Минфин оценил выпадающие доходы в 183 миллиарда рублей. Компенсировать их предполагалось за счет роста цен на крепкий алкоголь, но добиться этого не удалось."
results = nlp(text)
for entity in results:
    print(f"{entity['word']} -> {entity['entity_group']} (confidence: {entity['score']:.2f})")

Device set to use cpu


Правительство -> ORGANIZATION (confidence: 0.59)
России -> ORGANIZATION (confidence: 0.52)
на движимое -> LAW (confidence: 0.54)
с 1 января 2019 -> DATE (confidence: 0.77)
замминистра -> PROFESSION (confidence: 0.47)
Алексей -> PERSON (confidence: 0.80)
« -> ORGANIZATION (confidence: 0.51)
» -> PROFESSION (confidence: 0.63)
. Спикер Совета -> PROFESSION (confidence: 0.56)
Валентина -> PERSON (confidence: 0.73)
». В -> DATE (confidence: 0.54)
году -> ORGANIZATION (confidence: 0.37)
183 -> MONEY (confidence: 0.31)
